First download the model weights and parameters, and install caffe

In [1]:
import caffe

In [2]:
caffe.set_mode_cpu()

In [3]:
#load the model
net = caffe.Net('model/VGG_CNN_S_deploy.prototxt',
                'model/VGG_CNN_S.caffemodel',
                caffe.TEST)

In [4]:
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)
net.blobs['data'].reshape(1,3,224,224)
im = caffe.io.load_image('model/cat.jpg')
net.blobs['data'].data[...] = transformer.preprocess('data', im)

In [5]:
out = net.forward()

In [6]:
top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
print(top_k)

[331 285 287 281 330 278 277 282 335 332 937 333 998 284]


In [16]:
# generating train and test dataset txt files
import pandas as pd
import numpy as np
import os
def generateTexts():
    i=0
    j=0
    train = pd.DataFrame()
    labelss = pd.DataFrame()
    data = pd.read_csv('combined/haryana.csv')
    data = data.append(pd.read_csv('combined/punjab.csv'))
    print(len(data))
    for index, row in data.iterrows():
        filename = 'image/'+str(row['State'])+str(row['District'])+str(row['tehsil'])+str(row['village'])+'.PNG'
        if os.path.exists(filename):
            entry = []
            entry.append([filename, row['electricity']])
                          #,row['treated_water'],row['electronics'],row['phone'],row['transport']])
            train = train.append(entry)
            entry = []
            entry.append([row['electricity'],row['treated_water'],row['electronics'],row['phone'],row['transport']])
            labelss = labelss.append(entry)
    train.to_csv('data.txt', ' ', header = False, index = False)
    labelss.to_csv('labelss.txt', ' ', header = False, index = False)
    print(len(train))

In [17]:
generateTexts()

17864
5954


The model finally begins

In [3]:
import caffe

caffe.set_mode_cpu()

import numpy as np
import tempfile

ModuleNotFoundError: No module named 'caffe'

In [4]:
model_root = './model/'
import os
weights = os.path.join(model_root, 'VGG_CNN_S.caffemodel')
assert os.path.exists(weights)

In [5]:
solver = caffe.get_solver(model_root+'solver.prototxt')

NameError: name 'caffe' is not defined

In [6]:
solver.net.copy_from(weights)

NameError: name 'solver' is not defined

In [7]:
niter = 20000
solver.step(niter)
weight_dir = tempfile.mkdtemp()
filename = 'weights.mymodel.caffemodel'
dirs = os.path.join(weight_dir, filename)
solver.net.save(dirs)
del solver

NameError: name 'solver' is not defined

In [8]:
import pandas as pd
import numpy as np
import os
import scipy as sp
def generateLabels():
    train = pd.DataFrame()
    test = pd.DataFrame()
    data = pd.read_csv('combined/haryana.csv')
    data = data.append(pd.read_csv('combined/punjab.csv'))
    print(len(data))
    for index, row in data.iterrows():
        filename = 'image/'+str(row['State'])+str(row['District'])+str(row['tehsil'])+str(row['village'])+'.PNG'
        if os.path.exists(filename):
            entry = []
            entry.append([row['electricity'],row['treated_water'],row['electronics'],row['phone'],row['transport']])
            train = train.append(entry)
    train.to_csv('labelss.txt', header = False, index = False)
generateLabels()

17864


In [1]:
import numpy as np
mtt = np.loadtxt(open("labelss.txt", "rb"), delimiter=" ")

In [2]:
from scipy import io
io.savemat("labels.mat", {'labels': mtt})

In [2]:
mtt

array([[76.2       , 56.7       , 19.2       , 15.5       , 30.1       ],
       [72.9       , 64.5       , 26.8       , 73.7       , 39.9       ],
       [86.        , 10.1       , 14.73333333, 81.4       , 10.9       ],
       ...,
       [99.7       , 82.9       , 39.5       , 98.3       , 79.2       ],
       [91.3       , 24.6       , 31.76666667, 69.1       , 28.8       ],
       [98.6       , 48.6       , 36.26666667, 85.9       , 49.4       ]])